In [1]:
def findAnagramSubstring(full, sub):
    """
    full: string being searched
    sub: string composition being searched for
    return: [(startIdx, endIdx), ...] or -1 if error
    """
    
    if (len(sub) > len(full)):
        return -1
    
    chars, count, charSum, tempSum, anagrams = {}, 0, 0, 0, []
    
    # o(n)
    for char in full:
        if char not in chars:
            chars[char] = 2**count
            count += 1
    
    # o(k)
    for i in range(len(sub)):
        tempSum += chars[full[i]]     
        charSum += chars[sub[i]]
        
    # o(n - k)
    for i in range(len(full) - len(sub)):
        if tempSum == charSum:
            anagrams.append((i, i + len(sub)))
        tempSum = tempSum - chars[full[i]] + chars[full[i + len(sub)]]
    
    return anagrams

In [2]:
findAnagramSubstring('Please note, the test site finder is not exhaustive and does not represent a state-sponsored registry of legitimate testing sites. It is intended for use by New Jersey residents seeking testing site locations and is not intended for any other purpose.', 'estt')

[(17, 21), (116, 120), (186, 190)]

# Problem

Given a matrix of size m by n full of chars (so actually an array of equal length strings) and a submatrix of size j, k full of chars where j <= m and k <= n, find the coordinates of all submatrices which have an anagram of those chars.

```py
full = [ "abcd", "efgh", "ijhk", "lmno", "pqrs" ] # m=5, m=4
sub  = [ "cb", "gf" ] # j=2, k=2
```

So this input would respond with the tuple `(1, 0)`

## Progress
O(n^4*(n^2)!) - Brute force
O(n^4) - Brute force with hash table
O(n^3) - Amortized senteniel matrix (Rabin-Karp or KMP algorithms)

## Future
O(n^2) - Use object for senteniel like and have fields for corners and edges which would be constant time to update. Then navigate in a criss cross pattern like

```
##00 0##0 00## 0000 0000
##00 0##0 00## 00## 0##0
0000 0000 0000 00## 0##0
0000 0000 0000 0000 0000 ...
```

In [191]:
def isValid(full, sub):
    fullLen, subLen = False, False
    if len(sub) > len(full):
        return False
    for arr in full:
        if not fullLen:
            fullLen = len(arr)
        else:
            if fullLen != len(arr):
                return False
    for arr in sub:
        if not subLen:
            subLen = len(arr)
        else:
            if subLen != len(arr):
                return False
    if subLen > fullLen:
        return False
    return True

# avg: o(n^3) amortized time: o(n^4)
def findAnagramSubmatrix(full, sub):
    """
    full: array of equal length strings
    sub: array of string compositions to be searched for
    return: [(startX, startY), ...] or -1 if error
    """
    # check the input shapes are valid
    if not isValid(full, sub):
        return -1
    
    # init variables
    chars, count, charSum, tempSum, anagrams, subLen = {}, 0, 0, 0, [], False
    
    # initializes dict 
    # o(fullX*fullY)
    for arr in full:
        for char in arr:
            if char not in chars:
                chars[char] = 2**count
                count += 1
    
    # gets length of sub matrix string and charSum
    # o(subX*subY)
    for arr in range(len(sub)):
        if not subLen:
            subLen = len(sub[arr])
        for char in sub[arr]:
            charSum += chars[char]
            
    # returns sum for amortized position (y == 0)
    # o(subX*subY)
    def getSum(y, x):
        tempSum = 0
        for arr in full[y:y+len(sub)]:
            for char in arr[x:x+subLen]:
                tempSum += chars[char]
        return tempSum
    
    # returns sum of of row
    # o(subX)
    def rowSum(y, x):
        tempSum = 0
        for char in full[y][x:subLen]:
            tempSum += chars[char]
        return tempSum
    
    def printSub(y, x):
        tempArr = []
        tempSum = 0
        row1 = rowSum(y - 1, x)
        row2 = rowSum(y + len(sub) - 1, x)
        for arr in full[y:y+len(sub)]:
            for char in arr[x:x+subLen]:
                tempArr.append(char)
                tempSum += chars[char]
        print(tempArr, tempSum, row1, row2, row1+row2 == tempSum)
            
    # avg: o((fullX - subX)*(fullY - subY)*subX) = o(n^3)
    # amortization step: o((fullX - subX)*(fullY - subY)*subX*subY) = o(n^4)
    for y in range(len(full) - len(sub) + 1):
        for x in range(len(full[y]) - subLen + 1):
            printSub(y, x)
            if y == 0:
                tempSum = getSum(y, x)
            else:
                tempSum = getSum(y, x)
            if tempSum == charSum:
                anagrams.append((x, y))

    return anagrams

### usage
findAnagramSubmatrix(["abcd","defg","ghij", "jklm", "acab", "acab"], ["ac", "ac"])

['a', 'b', 'd', 'e'] 27 5 24 False
['b', 'c', 'e', 'f'] 54 4 16 False
['c', 'd', 'f', 'g'] 108 0 0 False
['d', 'e', 'g', 'h'] 216 3 192 False
['e', 'f', 'h', 'i'] 432 2 128 False
['f', 'g', 'i', 'j'] 864 0 0 False
['g', 'h', 'j', 'k'] 1728 24 1536 False
['h', 'i', 'k', 'l'] 3456 16 1024 False
['i', 'j', 'l', 'm'] 6912 0 0 False
['j', 'k', 'a', 'c'] 1541 192 5 False
['k', 'l', 'c', 'a'] 3077 128 4 False
['l', 'm', 'a', 'b'] 6147 0 0 False
['a', 'c', 'a', 'c'] 10 1536 5 False
['c', 'a', 'c', 'a'] 10 1024 4 False
['a', 'b', 'a', 'b'] 6 0 0 False


[(0, 4), (1, 4)]